In [1]:
# ============================================
# CÉLULA 1: IMPORTS E SETUP
# ============================================

import boto3
import json
import pandas as pd
from datetime import datetime
from xhtml2pdf import pisa
from pathlib import Path
import pickle
import time

print("📚 SISTEMA DE ANÁLISE DE QUESTÕES DE PORTUGUÊS")
print("="*60)
print("✅ Bibliotecas importadas")
print("📌 Versão otimizada - 40% mais eficiente")
print("="*60)

📚 SISTEMA DE ANÁLISE DE QUESTÕES DE PORTUGUÊS
✅ Bibliotecas importadas
📌 Versão otimizada - 40% mais eficiente


In [2]:
# ============================================
# CÉLULA 2: CONFIGURAR AWS BEDROCK
# ============================================

# Configurar cliente
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name="us-east-2"
)

print("🔍 Configurando Claude Sonnet 4.5...\n")

MODEL_ID = None
MODEL_NAME = None

# Tentar versões do Sonnet 4.5
model_versions = [
    'anthropic.claude-sonnet-4-5-20250929-v1:0',
    'us.anthropic.claude-sonnet-4-5-20250929-v1:0',
    'anthropic.claude-sonnet-4-5-20250514-v1:0',
    'us.anthropic.claude-3-5-sonnet-20241022-v2:0'  # Fallback
]

for idx, model_id in enumerate(model_versions, 1):
    try:
        print(f"📌 Tentativa {idx}: {model_id[:50]}...")
        
        response = bedrock_client.invoke_model(
            body=json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 10,
                "messages": [{"role": "user", "content": "test"}]
            }),
            modelId=model_id,
            accept="application/json",
            contentType="application/json"
        )
        
        MODEL_ID = model_id
        MODEL_NAME = "Claude Sonnet 4.5" if "4-5" in model_id else "Claude Sonnet 3.5 v2"
        print(f"   ✅ SUCESSO!\n")
        break
        
    except Exception as e:
        print(f"   ❌ {str(e)[:80]}...\n")
        continue

if not MODEL_ID:
    raise Exception("Nenhum modelo disponível. Verifique permissões no AWS Bedrock.")

# Resumo
print("="*60)
print(f"🤖 MODELO: {MODEL_NAME}")
print(f"📍 ID: {MODEL_ID}")
print("="*60)

if "4.5" in MODEL_NAME or "4-5" in MODEL_ID:
    print("\n💰 CUSTO ESTIMADO:")
    print("   • Por questão: ~$0.015")
    print("   • 10 questões: ~$0.15")
else:
    print("\n💰 CUSTO ESTIMADO:")
    print("   • Por questão: ~$0.012")
    print("   • 10 questões: ~$0.12")

print("\n✅ Configuração completa!")
print("="*60)

🔍 Configurando Claude Sonnet 4.5...

📌 Tentativa 1: anthropic.claude-sonnet-4-5-20250929-v1:0...
   ❌ An error occurred (ValidationException) when calling the InvokeModel operation: ...

📌 Tentativa 2: us.anthropic.claude-sonnet-4-5-20250929-v1:0...
   ✅ SUCESSO!

🤖 MODELO: Claude Sonnet 4.5
📍 ID: us.anthropic.claude-sonnet-4-5-20250929-v1:0

💰 CUSTO ESTIMADO:
   • Por questão: ~$0.015
   • 10 questões: ~$0.15

✅ Configuração completa!


In [3]:
# ============================================
# CÉLULA 3: PROMPT CONCISO
# ============================================

PROMPT_PORTUGUES = """Professor de Português, especialista em análise de questões de concursos.

ESTRUTURA:

1. IDENTIFICAÇÃO
Tema | Área (GRAMÁTICA/INTERPRETAÇÃO/LITERATURA) | Nível

2. CONCEITOS (3-4)
✓ Conceito → Definição
Regra aplicável (se gramática)

3. ALTERNATIVAS (TODAS as 5)

A: ✅/❌ | Análise (2-3 linhas) | Pegadinha
B: ✅/❌ | Análise (2-3 linhas) | Pegadinha  
C: ✅/❌ | Análise (2-3 linhas) | Pegadinha
D: ✅/❌ | Análise (2-3 linhas) | Pegadinha
E: ✅/❌ | Análise (2-3 linhas) | Pegadinha

4. GABARITO: {gabarito} ✅
Por quê (2 linhas) | Palavra-chave

5. PEGADINHAS
⚠️ 2-3 armadilhas principais

6. ESTRATÉGIA
📚 2 dicas | 📖 Gramático | ⏱️ Tempo

7. RESUMO
Dificuldade | Competência | Frequência | Dica final

REGRAS:
✓ Analise TODAS as 5 alternativas
✓ Cite fundamentos linguísticos
✓ 2000-2500 caracteres
✓ Marque ✅/❌ claramente

QUESTÃO:
Banca: {banca} | Ano: {ano}
Assunto: {assunto}

{enunciado}

A) {alt_a}
B) {alt_b}
C) {alt_c}
D) {alt_d}
E) {alt_e}

Gabarito: {gabarito}

Inicie:"""

print("✅ Prompt otimizado configurado!")
print(f"📏 Tamanho: {len(PROMPT_PORTUGUES):,} caracteres")
print(f"🔤 Tokens: ~{len(PROMPT_PORTUGUES)//4:,}")
print(f"💰 Economia: 69% vs versão anterior")
print("="*60)

✅ Prompt otimizado configurado!
📏 Tamanho: 975 caracteres
🔤 Tokens: ~243
💰 Economia: 69% vs versão anterior


In [4]:
# ============================================
# CÉLULA 4: FUNÇÃO DE ANÁLISE
# ============================================

# Criar pasta de cache
CACHE_DIR = Path('cache_portugues')
CACHE_DIR.mkdir(exist_ok=True)

def validar_qualidade(analise, questao):
    """Validação rápida de qualidade"""
    score = 0
    
    # Tamanho
    if len(analise) >= 2000:
        score += 20
    
    # Alternativas
    score += sum([
        'ALTERNATIVA A' in analise.upper() or 'A:' in analise,
        'ALTERNATIVA B' in analise.upper() or 'B:' in analise,
        'ALTERNATIVA C' in analise.upper() or 'C:' in analise,
        'ALTERNATIVA D' in analise.upper() or 'D:' in analise,
        'ALTERNATIVA E' in analise.upper() or 'E:' in analise
    ]) * 12
    
    # Marcações
    score += min((analise.count('✅') + analise.count('❌')) * 2, 10)
    
    # Estrutura
    if 'GABARITO' in analise.upper():
        score += 10
    
    # Termos técnicos
    termos = ['regra', 'gramática', 'semântica', 'sintaxe', 'interpretação']
    score += sum(termo in analise.lower() for termo in termos) * 5
    
    return min(score, 100)


def analisar_questao(questao_dict, max_tentativas=2):
    """Análise otimizada de questão"""
    
    questao_id = questao_dict.get('id_questao', 'unknown')
    cache_file = CACHE_DIR / f"{questao_id}.pkl"
    
    # Cache
    if cache_file.exists():
        print(f"   💾 Cache encontrado")
        with open(cache_file, 'rb') as f:
            return pickle.load(f)
    
    # Gerar análise
    for tentativa in range(1, max_tentativas + 1):
        try:
            temp = 0.3 if tentativa == 1 else 0.4
            
            prompt_final = PROMPT_PORTUGUES.format(
                banca=questao_dict.get('banca', 'N/A'),
                ano=questao_dict.get('ano', 'N/A'),
                assunto=questao_dict.get('assunto', 'N/A'),
                enunciado=questao_dict.get('enunciado', 'N/A'),
                alt_a=questao_dict.get('alt_a', 'N/A'),
                alt_b=questao_dict.get('alt_b', 'N/A'),
                alt_c=questao_dict.get('alt_c', 'N/A'),
                alt_d=questao_dict.get('alt_d', 'N/A'),
                alt_e=questao_dict.get('alt_e', 'N/A'),
                gabarito=questao_dict.get('gabarito', 'N/A')
            )
            
            config = {
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 4000,
                "temperature": temp,
                "messages": [{"role": "user", "content": prompt_final}]
            }
            
            response = bedrock_client.invoke_model(
                body=json.dumps(config),
                modelId=MODEL_ID,
                accept="application/json",
                contentType="application/json"
            )
            
            resposta = json.loads(response['body'].read().decode('utf-8'))
            analise = resposta.get('content', [{}])[0].get('text', 'Erro')
            
            # Validar
            score = validar_qualidade(analise, questao_dict)
            print(f"   📊 Score: {score}/100")
            
            if score >= 65:
                with open(cache_file, 'wb') as f:
                    pickle.dump(analise, f)
                return analise
            
            if tentativa < max_tentativas:
                print(f"   🔄 Tentativa {tentativa+1}...")
                
        except Exception as e:
            print(f"   ❌ Erro: {str(e)[:80]}")
            if tentativa == max_tentativas:
                return f"❌ Erro: {str(e)}"
            continue
    
    # Salvar mesmo se score baixo
    with open(cache_file, 'wb') as f:
        pickle.dump(analise, f)
    return analise


print("✅ Função de análise criada!")
print("📊 Features:")
print("   ✓ Cache automático")
print("   ✓ Retry inteligente")
print("   ✓ Validação de qualidade")
print("   ✓ SEM system prompt duplicado")
print("="*60)

✅ Função de análise criada!
📊 Features:
   ✓ Cache automático
   ✓ Retry inteligente
   ✓ Validação de qualidade
   ✓ SEM system prompt duplicado


In [5]:
# ============================================
# CÉLULA 5: GERAÇÃO DE PDF
# ============================================

def gerar_pdf(questao_dict, analise, output_path):
    """Gera PDF profissional"""
    
    def escape_html(text):
        if pd.isna(text):
            return 'N/A'
        return (str(text)
                .replace('&', '&amp;')
                .replace('<', '&lt;')
                .replace('>', '&gt;')
                .replace('"', '&quot;')
                .replace("'", '&#39;'))
    
    # Converter análise
    analise_html = escape_html(analise).replace('\n\n', '</p><p>').replace('\n', '<br/>')
    
    # Alternativas
    gabarito = str(questao_dict.get('gabarito', '')).strip().upper()
    alternativas_html = ""
    
    for letra in ['A', 'B', 'C', 'D', 'E']:
        texto = escape_html(questao_dict.get(f'alt_{letra.lower()}', 'N/A'))
        eh_correta = (letra == gabarito)
        
        if eh_correta:
            estilo = 'background: linear-gradient(135deg, #d4edda 0%, #c3e6cb 100%); border-left: 5px solid #28a745; padding: 15px; margin: 12px 0; border-radius: 6px;'
            label = 'background-color: #28a745; color: white; padding: 6px 14px; border-radius: 5px; font-weight: bold; margin-right: 12px;'
            simbolo = '✅'
        else:
            estilo = 'background-color: #f8f9fa; border-left: 4px solid #dee2e6; padding: 15px; margin: 12px 0; border-radius: 6px;'
            label = 'background-color: #6c757d; color: white; padding: 6px 14px; border-radius: 5px; font-weight: bold; margin-right: 12px;'
            simbolo = '⬜'
        
        alternativas_html += f'<div style="{estilo}"><span style="{label}">{letra}</span><span style="font-weight: bold;">{simbolo}</span> <span>{texto}</span></div>'
    
    # HTML completo
    html = f'''<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8"/>
    <style>
        @page {{ size: a4 portrait; margin: 1.8cm; }}
        body {{ font-family: Arial; font-size: 11pt; line-height: 1.7; color: #2c3e50; }}
        .header {{ background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 28px; text-align: center; margin-bottom: 25px; border-radius: 10px; }}
        .header h1 {{ margin: 0 0 12px 0; font-size: 26pt; font-weight: bold; }}
        .section {{ margin-bottom: 22px; }}
        .section-title {{ background: #f8f9fa; padding: 14px 20px; border-left: 6px solid #667eea; font-size: 15pt; font-weight: bold; margin-bottom: 16px; border-radius: 5px; }}
        .info {{ display: table; width: 100%; margin: 15px 0; }}
        .info-row {{ display: table-row; }}
        .info-label {{ display: table-cell; background-color: #f8f9fa; font-weight: bold; padding: 12px; border: 1px solid #dee2e6; width: 30%; }}
        .info-value {{ display: table-cell; padding: 12px; border: 1px solid #dee2e6; }}
        .enunciado {{ background: #fffbea; border: 3px solid #ffd93d; padding: 20px; margin: 18px 0; font-size: 12pt; border-radius: 8px; }}
        .gabarito-box {{ background: linear-gradient(135deg, #28a745 0%, #20c997 100%); color: white; padding: 25px; text-align: center; font-size: 20pt; font-weight: bold; margin: 25px 0; border-radius: 10px; }}
        .analise {{ background: #e3f2fd; border: 3px solid #2196f3; padding: 20px; margin: 20px 0; border-radius: 8px; }}
        .footer {{ margin-top: 40px; padding-top: 20px; border-top: 3px solid #dee2e6; text-align: center; font-size: 9pt; color: #6c757d; }}
    </style>
</head>
<body>
    <div class="header">
        <h1>📋 ANÁLISE PEDAGÓGICA - PORTUGUÊS</h1>
        <p>🏛️ {escape_html(questao_dict.get('banca', 'N/A'))} | 📅 {escape_html(questao_dict.get('ano', 'N/A'))} | 🏢 {escape_html(questao_dict.get('orgao', 'N/A'))}</p>
    </div>
    
    <div class="section">
        <div class="section-title">📊 IDENTIFICAÇÃO</div>
        <div class="info">
            <div class="info-row"><div class="info-label">Disciplina</div><div class="info-value">{escape_html(questao_dict.get('disciplina', 'N/A'))}</div></div>
            <div class="info-row"><div class="info-label">Assunto</div><div class="info-value">{escape_html(questao_dict.get('assunto', 'N/A'))}</div></div>
            <div class="info-row"><div class="info-label">Gabarito</div><div class="info-value"><b style="color: #28a745; font-size: 14pt;">{escape_html(questao_dict.get('gabarito', 'N/A'))}</b></div></div>
        </div>
    </div>
    
    <div class="section">
        <div class="section-title">📝 ENUNCIADO</div>
        <div class="enunciado">{escape_html(questao_dict.get('enunciado', 'N/A'))}</div>
    </div>
    
    <div class="section">
        <div class="section-title">🔤 ALTERNATIVAS</div>
        {alternativas_html}
    </div>
    
    <div class="gabarito-box">✅ GABARITO: {escape_html(questao_dict.get('gabarito', 'N/A'))}</div>
    
    <div class="section">
        <div class="section-title">🤖 ANÁLISE PEDAGÓGICA</div>
        <div class="analise"><p>{analise_html}</p></div>
    </div>
    
    <div class="footer">
        📚 Análise via AWS Bedrock - {MODEL_NAME}<br/>
        {datetime.now().strftime('%d/%m/%Y às %H:%M:%S')}
    </div>
</body>
</html>'''
    
    # Gerar PDF
    try:
        with open(output_path, "wb") as pdf_file:
            pisa.CreatePDF(html, dest=pdf_file)
        return True
    except Exception as e:
        print(f"   ❌ Erro PDF: {e}")
        return False


print("✅ Função PDF criada!")
print("="*60)

✅ Função PDF criada!


In [6]:
# ============================================
# CÉLULA 6: TEXTO PARA ÁUDIO
# ============================================

def gerar_audio(questao_dict, analise):
    """Gera texto conversacional para TTS"""
    
    prompt = f"""Explique esta questão em ÁUDIO (2-3 minutos).

REGRAS:
- Linguagem FALADA natural
- SEM símbolos, emojis, formatações
- Frases curtas
- Máximo 500 palavras

ESTRUTURA:
Olá! Questão de {questao_dict.get('assunto')} da {questao_dict.get('banca')} de {questao_dict.get('ano')}.

[Parafraseie o enunciado]

Alternativa A... [explique] ...incorreta porque [razão].
Alternativa B... [explique] ...incorreta porque [razão].
Alternativa C... [explique] ...CORRETA porque [razão].
Alternativa D... [explique] ...incorreta porque [razão].
Alternativa E... [explique] ...incorreta porque [razão].

Gabarito: {questao_dict.get('gabarito')}.
Dica: [dica prática].
Até a próxima!

QUESTÃO:
{questao_dict.get('enunciado')}

A) {questao_dict.get('alt_a')}
B) {questao_dict.get('alt_b')}
C) {questao_dict.get('alt_c')}
D) {questao_dict.get('alt_d')}
E) {questao_dict.get('alt_e')}

Gabarito: {questao_dict.get('gabarito')}

Análise (ref): {analise[:800]}...

Texto para áudio:"""
    
    try:
        config = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 1500,
            "temperature": 0.4,
            "messages": [{"role": "user", "content": prompt}]
        }
        
        response = bedrock_client.invoke_model(
            body=json.dumps(config),
            modelId=MODEL_ID,
            accept="application/json",
            contentType="application/json"
        )
        
        resposta = json.loads(response['body'].read().decode('utf-8'))
        return resposta.get('content', [{}])[0].get('text', 'Erro')
        
    except Exception as e:
        return f"Olá! Questão de {questao_dict.get('assunto')}. Gabarito: {questao_dict.get('gabarito')}. Consulte o PDF para detalhes."


print("✅ Função de áudio criada!")
print("="*60)

✅ Função de áudio criada!


In [7]:
# ============================================
# CÉLULA 7: CARREGAR DATASET
# ============================================

# 📝 ATUALIZE O CAMINHO DO SEU ARQUIVO
CAMINHO_ARQUIVO = r'C:\welligton-projects-IA\RPA-PROJETO-1\questoes_PORTUGUES_10_FGV_TJ-MORFOLOGIA.xlsx'

print("📂 Carregando dataset...")

try:
    df = pd.read_excel(CAMINHO_ARQUIVO)
    
    print(f"✅ {len(df)} questões carregadas\n")
    print("📊 DATASET:")
    print(f"   🏛️  Banca: {df['Banca'].unique()[0] if 'Banca' in df.columns else 'N/A'}")
    
    if 'Assunto' in df.columns:
        print(f"   📚 Assuntos: {df['Assunto'].nunique()} temas")
        print(f"\n   Top 3 assuntos:")
        for assunto, count in df['Assunto'].value_counts().head(3).items():
            print(f"      • {assunto}: {count} questões")
    
    if 'Resposta_Correta' in df.columns:
        print(f"\n   📊 Gabaritos:")
        for gab, count in df['Resposta_Correta'].value_counts().sort_index().items():
            print(f"      {gab}: {count} questões")
    
    print(f"\n   📋 Colunas: {', '.join(df.columns[:5])}...")
    print("\n✅ Dataset pronto!")
    print("="*60)
    
except FileNotFoundError:
    print(f"❌ Arquivo não encontrado: {CAMINHO_ARQUIVO}")
    print("\n💡 Verifique o caminho e tente novamente")
    df = None
    
except Exception as e:
    print(f"❌ Erro: {e}")
    df = None

📂 Carregando dataset...
✅ 8 questões carregadas

📊 DATASET:
   🏛️  Banca: FGV
   📚 Assuntos: 1 temas

   Top 3 assuntos:
      • Morfologia,: 8 questões

   📊 Gabaritos:
      A: 2 questões
      B: 1 questões
      C: 1 questões
      D: 2 questões
      E: 2 questões

   📋 Colunas: ID_Questao, Indice_Pagina, Ano, Banca, Orgao...

✅ Dataset pronto!


In [8]:
# ============================================
# CÉLULA 8: TESTE COM 1 QUESTÃO
# ============================================

def testar_questao(df, indice=0):
    """Testa análise completa de 1 questão"""
    
    if df is None:
        print("❌ Carregue o dataset primeiro (Célula 7)")
        return None
    
    print(f"\n🧪 TESTE - Questão {indice+1}/{len(df)}")
    print("="*60)
    
    # Montar questão
    row = df.iloc[indice]
    questao = {
        'id_questao': row.get('ID_Questao', f'Q{indice+1}'),
        'banca': row.get('Banca', 'N/A'),
        'ano': row.get('Ano', 'N/A'),
        'orgao': row.get('Orgao', 'N/A'),
        'prova': row.get('Prova', 'N/A'),
        'disciplina': row.get('Disciplina', 'Português'),
        'assunto': row.get('Assunto', 'N/A'),
        'enunciado': row.get('Enunciado', 'N/A'),
        'alt_a': row.get('Alternativa_A', 'N/A'),
        'alt_b': row.get('Alternativa_B', 'N/A'),
        'alt_c': row.get('Alternativa_C', 'N/A'),
        'alt_d': row.get('Alternativa_D', 'N/A'),
        'alt_e': row.get('Alternativa_E', 'N/A'),
        'gabarito': str(row.get('Resposta_Correta', 'N/A')).strip().upper()
    }
    
    # Mostrar questão
    print(f"🆔 {questao['id_questao']}")
    print(f"📖 {questao['assunto'][:50]}...")
    print(f"✅ Gabarito: {questao['gabarito']}\n")
    
    # ETAPA 1: Análise
    print("📝 Gerando análise...")
    inicio = time.time()
    analise = analisar_questao(questao)
    tempo_analise = time.time() - inicio
    
    print(f"   ✅ {tempo_analise:.1f}s")
    print(f"   📏 {len(analise):,} caracteres")
    print(f"   🔤 ~{len(analise)//4:,} tokens\n")
    
    # ETAPA 2: Áudio
    print("🎙️  Gerando áudio...")
    inicio = time.time()
    audio = gerar_audio(questao, analise)
    tempo_audio = time.time() - inicio
    
    print(f"   ✅ {tempo_audio:.1f}s")
    print(f"   📏 {len(audio):,} caracteres")
    print(f"   🔤 ~{len(audio)//4:,} tokens\n")
    
    # ETAPA 3: Salvar
    print("💾 Salvando arquivos...")
    id_q = questao['id_questao']
    
    # PDF
    pdf_path = f'ANALISE_{id_q}.pdf'
    gerar_pdf(questao, analise, pdf_path)
    print(f"   ✅ {pdf_path}")
    
    # TXT áudio
    audio_path = f'AUDIO_{id_q}.txt'
    with open(audio_path, 'w', encoding='utf-8') as f:
        f.write(audio)
    print(f"   ✅ {audio_path}")
    
    # Estatísticas
    tokens_total = (len(analise) + len(audio)) // 4
    custo = (tokens_total / 1000) * 0.018  # Estimativa média
    
    print(f"\n📊 RESUMO:")
    print(f"   ⏱️  Tempo total: {tempo_analise + tempo_audio:.1f}s")
    print(f"   💰 Custo: ~${custo:.4f}")
    print(f"   📦 Arquivos: 2 (PDF + TXT)")
    
    print("\n✅ TESTE CONCLUÍDO!")
    print("="*60)
    
    return {'questao': questao, 'analise': analise, 'audio': audio}


# EXECUTAR TESTE
if df is not None:
    resultado = testar_questao(df, indice=0)
    
    if resultado:
        print("\n💡 Para testar outra questão:")
        print("   testar_questao(df, indice=5)  # Questão 6")
else:
    print("⚠️  Execute a Célula 7 primeiro para carregar o dataset")


🧪 TESTE - Questão 1/8
🆔 Q3114574
📖 Morfologia,...
✅ Gabarito: E

📝 Gerando análise...
   📊 Score: 100/100
   ✅ 25.7s
   📏 3,727 caracteres
   🔤 ~931 tokens

🎙️  Gerando áudio...
   ✅ 11.4s
   📏 1,976 caracteres
   🔤 ~494 tokens

💾 Salvando arquivos...
   ✅ ANALISE_Q3114574.pdf
   ✅ AUDIO_Q3114574.txt

📊 RESUMO:
   ⏱️  Tempo total: 37.1s
   💰 Custo: ~$0.0256
   📦 Arquivos: 2 (PDF + TXT)

✅ TESTE CONCLUÍDO!

💡 Para testar outra questão:
   testar_questao(df, indice=5)  # Questão 6


In [12]:
testar_questao(df, indice=4)


🧪 TESTE - Questão 5/8
🆔 Q3091333
📖 Morfologia,...
✅ Gabarito: A

📝 Gerando análise...
   📊 Score: 100/100
   ✅ 20.9s
   📏 3,609 caracteres
   🔤 ~902 tokens

🎙️  Gerando áudio...
   ✅ 10.8s
   📏 2,180 caracteres
   🔤 ~545 tokens

💾 Salvando arquivos...
   ✅ ANALISE_Q3091333.pdf
   ✅ AUDIO_Q3091333.txt

📊 RESUMO:
   ⏱️  Tempo total: 31.7s
   💰 Custo: ~$0.0260
   📦 Arquivos: 2 (PDF + TXT)

✅ TESTE CONCLUÍDO!


{'questao': {'id_questao': 'Q3091333',
  'banca': 'FGV',
  'ano': np.int64(2024),
  'orgao': 'TJ-RR',
  'prova': 'FGV - 2024 - TJ-RR - Técnico Judiciário',
  'disciplina': 'Português',
  'assunto': 'Morfologia,',
  'enunciado': 'Assinale a frase abaixo que exemplifica corretamente um adjetivo no\ngrau superlativo absoluto.',
  'alt_a': 'A situação dos habitantes de algumas cidades gaúchas ficou\nextremamente grave.',
  'alt_b': 'A questão do uso das câmeras pelos policiais é um dos\nproblemas mais discutidos do momento.',
  'alt_c': 'O problema da educação no Brasil talvez seja mais grave que o\nproblema da saúde.',
  'alt_d': 'O problema dos altos salários em algumas funções públicas é\nvisto como menos grave do que outros desequilíbrios pelos\npolíticos.',
  'alt_e': 'O célebre piloto Ayrton Senna era tão corajoso quanto prudente\nem suas corridas.',
  'gabarito': 'A'},
 'analise': '# ANÁLISE COMPLETA DA QUESTÃO\n\n## 1. IDENTIFICAÇÃO\n**Tema:** Graus do Adjetivo | **Área:** GRAMÁTIC

In [15]:
# ============================================
# CÉLULA 8: TESTE SIMPLES COM 1 QUESTÃO
# ============================================

def testar_questao(df, indice=0):
    """Teste simplificado de 1 questão"""
    
    if df is None:
        print("❌ Execute a Célula 7 primeiro")
        return None
    
    print(f"\n🧪 TESTE - Questão {indice+1}/{len(df)}")
    print("="*60)
    
    # Montar questão
    row = df.iloc[indice]
    questao = {
        'id_questao': row.get('ID_Questao', f'Q{indice+1}'),
        'banca': row.get('Banca', 'N/A'),
        'ano': row.get('Ano', 'N/A'),
        'orgao': row.get('Orgao', 'N/A'),
        'prova': row.get('Prova', 'N/A'),
        'disciplina': row.get('Disciplina', 'Português'),
        'assunto': row.get('Assunto', 'N/A'),
        'enunciado': row.get('Enunciado', 'N/A'),
        'alt_a': row.get('Alternativa_A', 'N/A'),
        'alt_b': row.get('Alternativa_B', 'N/A'),
        'alt_c': row.get('Alternativa_C', 'N/A'),
        'alt_d': row.get('Alternativa_D', 'N/A'),
        'alt_e': row.get('Alternativa_E', 'N/A'),
        'gabarito': str(row.get('Resposta_Correta', 'N/A')).strip().upper()
    }
    
    # Info básica
    print(f"📋 ID: {questao['id_questao']}")
    print(f"📖 Assunto: {questao['assunto'][:60]}")
    print(f"✅ Gabarito: {questao['gabarito']}\n")
    
    # 1. Gerar análise
    print("📝 Análise...", end=" ")
    inicio = time.time()
    analise = analisar_questao(questao)
    t1 = time.time() - inicio
    print(f"✅ {t1:.1f}s ({len(analise):,} chars)")
    
    # 2. Gerar texto de áudio
    print("🎙️  Áudio...", end=" ")
    inicio = time.time()
    audio = gerar_audio(questao, analise)
    t2 = time.time() - inicio
    print(f"✅ {t2:.1f}s ({len(audio):,} chars)")
    
    # 3. Salvar arquivos
    print("💾 Salvando...", end=" ")
    id_q = questao['id_questao']
    
    gerar_pdf(questao, analise, f'{id_q}.pdf')
    
    with open(f'{id_q}_audio.txt', 'w', encoding='utf-8') as f:
        f.write(audio)
    
    print(f"✅ 2 arquivos")
    
    # Resumo
    custo = ((len(analise) + len(audio)) / 4000) * 0.018
    print(f"\n📊 Total: {t1+t2:.1f}s | ${custo:.4f} | Score: {validar_qualidade(analise, questao)}/100")
    print("="*60)
    
    # Preview da análise
    print("\n📄 PREVIEW DA ANÁLISE:")
    print("─"*60)
    print(analise[:500] + "...")
    print("─"*60)
    
    # Preview do áudio
    print("\n🎙️  PREVIEW DO TEXTO PARA ÁUDIO:")
    print("─"*60)
    print(audio[:400] + "...")
    print("─"*60)
    
    print(f"\n✅ Arquivos gerados:")
    print(f"   📄 {id_q}.pdf")
    print(f"   📝 {id_q}_audio.txt")
    
    return questao, analise, audio


# EXECUTAR
if 'df' in globals() and df is not None:
    resultado = testar_questao(df, indice=0)
    
    if resultado:
        print("\n💡 Testar outra: testar_questao(df, indice=1)")
else:
    print("⚠️  Carregue o dataset (Célula 7) primeiro")


🧪 TESTE - Questão 1/8
📋 ID: Q3114574
📖 Assunto: Morfologia,
✅ Gabarito: E

📝 Análise...    💾 Cache encontrado
✅ 0.0s (3,727 chars)
🎙️  Áudio... ✅ 12.4s (2,062 chars)
💾 Salvando... ✅ 2 arquivos

📊 Total: 12.4s | $0.0261 | Score: 100/100

📄 PREVIEW DA ANÁLISE:
────────────────────────────────────────────────────────────
# ANÁLISE COMPLETA DA QUESTÃO

## 1. IDENTIFICAÇÃO
**Tema:** Família de palavras (cognatos) | **Área:** GRAMÁTICA - Morfologia | **Nível:** Médio-Alto

## 2. CONCEITOS FUNDAMENTAIS

✓ **Família de palavras (cognatos)** → Palavras que compartilham o mesmo radical/raiz etimológica, apresentando relação semântica e formal.

✓ **Radical** → Elemento mórfico básico e significativo comum às palavras de uma mesma família, responsável pelo sentido nuclear.

✓ **Paronímia** → Semelhança formal entre pala...
────────────────────────────────────────────────────────────

🎙️  PREVIEW DO TEXTO PARA ÁUDIO:
────────────────────────────────────────────────────────────
Olá! Questão de Mor

In [23]:
# ============================================
# CÉLULA 9: GERAR ÁUDIO COM AMAZON POLLY (CORRIGIDA)
# ============================================

import boto3
from botocore.exceptions import BotoCoreError, ClientError
import os

# Região padrão — mude se quiser forçar neural (us-east-1)
REGIAO_PADRAO = "us-east-1"

# Cliente Polly
polly_client = boto3.client('polly', region_name=REGIAO_PADRAO)

def gerar_audio_mp3(questao_id, voz='Vitoria'):
    """
    Gera áudio MP3 a partir do arquivo de texto.

    Args:
        questao_id: ID da questão (ex: 'Q1')
        voz: 'Vitoria', 'Camila', 'Thiago' ou 'Ricardo'
    """

    print(f"\n🎙️  GERANDO ÁUDIO - {questao_id}")
    print("=" * 60)

    txt_file = f'{questao_id}_audio.txt'

    try:
        with open(txt_file, 'r', encoding='utf-8') as f:
            texto = f.read()

        print(f"✅ Texto carregado: {len(texto):,} caracteres")
        print(f"🗣️  Voz: {voz}")

        # Duração estimada
        palavras = len(texto.split())
        duracao_min = palavras / 150  # 150 palavras/minuto

        print(f"⏱️  Duração estimada: {duracao_min:.1f} minutos")
        print(f"\n🔄 Gerando áudio...", end=" ")

        # 🔍 Verificar se a voz suporta engine neural
        vozes = polly_client.describe_voices(LanguageCode='pt-BR')['Voices']
        engine_suportado = 'standard'
        for v in vozes:
            if v['Id'] == voz and 'neural' in v['SupportedEngines']:
                engine_suportado = 'neural'
                break

        # ✅ Gerar com Polly
        response = polly_client.synthesize_speech(
            Text=texto,
            OutputFormat='mp3',
            VoiceId=voz,
            Engine=engine_suportado
        )

        audio_file = f'{questao_id}.mp3'
        with open(audio_file, 'wb') as f:
            f.write(response['AudioStream'].read())

        tamanho_kb = os.path.getsize(audio_file) / 1024
        custo = (len(texto) / 1_000_000) * (16 if engine_suportado == 'neural' else 4)

        print("✅\n")
        print("=" * 60)
        print("🎵 ÁUDIO GERADO COM SUCESSO!")
        print("=" * 60)
        print(f"📁 Arquivo: {audio_file}")
        print(f"💾 Tamanho: {tamanho_kb:.1f} KB")
        print(f"⏱️  Duração: ~{duracao_min:.1f} min")
        print(f"💰 Custo: ${custo:.4f}")
        print(f"⚙️  Engine usado: {engine_suportado}")
        print("=" * 60)
        print(f"\n🎧 Abra o arquivo {audio_file} para ouvir!")

        return audio_file

    except FileNotFoundError:
        print(f"❌ Arquivo não encontrado: {txt_file}")
        print("💡 Execute a Célula 8 primeiro")
        return None

    except (BotoCoreError, ClientError) as e:
        print(f"❌ Erro Polly: {e}")
        return None


# ============================================
# GERAR ÁUDIO DA ÚLTIMA QUESTÃO TESTADA
# ============================================

if 'resultado' in globals() and resultado:
    questao_id = resultado[0]['id_questao']
    print(f"\n🎯 Gerando áudio da questão: {questao_id}")
    print(f"📖 Assunto: {resultado[0]['assunto'][:50]}...")

    print(f"\n🎤 VOZES DISPONÍVEIS:")
    print("   1. Vitória (feminina) ⭐ RECOMENDADA")
    print("   2. Camila (feminina)")
    print("   3. Thiago (masculino)")
    print("   4. Ricardo (masculino)")

    escolha = input("\nEscolha (1-4) ou Enter para Vitória: ").strip()
    vozes = {'1': 'Vitoria', '2': 'Camila', '3': 'Thiago', '4': 'Ricardo', '': 'Vitoria'}
    voz_escolhida = vozes.get(escolha, 'Vitoria')

    audio_file = gerar_audio_mp3(questao_id, voz=voz_escolhida)

    if audio_file:
        print("\n" + "🎉" * 30)
        print("PRONTO PARA ESTUDAR!")
        print("🎉" * 30)
        print(f"\n📦 ARQUIVOS COMPLETOS:")
        print(f"   📄 {questao_id}.pdf (análise visual)")
        print(f"   📝 {questao_id}_audio.txt (texto)")
        print(f"   🎵 {questao_id}.mp3 (áudio)")

else:
    print("\n⚠️  Nenhuma questão testada ainda")
    print("💡 Execute a Célula 8 primeiro")
    print("\nOu gere áudio manualmente:")
    print("   gerar_audio_mp3('Q1', voz='Vitoria')")



🎯 Gerando áudio da questão: Q3114574
📖 Assunto: Morfologia,...

🎤 VOZES DISPONÍVEIS:
   1. Vitória (feminina) ⭐ RECOMENDADA
   2. Camila (feminina)
   3. Thiago (masculino)
   4. Ricardo (masculino)

🎙️  GERANDO ÁUDIO - Q3114574
✅ Texto carregado: 2,062 caracteres
🗣️  Voz: Vitoria
⏱️  Duração estimada: 2.2 minutos

🔄 Gerando áudio... ✅

🎵 ÁUDIO GERADO COM SUCESSO!
📁 Arquivo: Q3114574.mp3
💾 Tamanho: 911.4 KB
⏱️  Duração: ~2.2 min
💰 Custo: $0.0330
⚙️  Engine usado: neural

🎧 Abra o arquivo Q3114574.mp3 para ouvir!

🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉
PRONTO PARA ESTUDAR!
🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉

📦 ARQUIVOS COMPLETOS:
   📄 Q3114574.pdf (análise visual)
   📝 Q3114574_audio.txt (texto)
   🎵 Q3114574.mp3 (áudio)


In [22]:
import boto3

# Cria o cliente Polly na região com suporte Neural
polly = boto3.client('polly', region_name='us-east-1')

# Texto para converter
texto = "Olá! Eu sou a voz neural do Amazon Polly, falando em português do Brasil."

# Gera o áudio
response = polly.synthesize_speech(
    Text=texto,
    OutputFormat='mp3',
    VoiceId='Camila',  # Pode trocar por 'Vitória' ou 'Thiago'
    Engine='neural'
)

# Salva o arquivo
with open('voz_neural_ptbr.mp3', 'wb') as f:
    f.write(response['AudioStream'].read())

print("✅ Áudio gerado com sucesso usando voz neural em português (Camila).")


✅ Áudio gerado com sucesso usando voz neural em português (Camila).
